In [1]:
import sys
import determinator
import nafigator
import stanza
import os
import logging

In [2]:
logging.basicConfig(stream=sys.stdout,
                    format='%(asctime)s %(message)s',
                    level=logging.INFO)
nlp = {"nl": stanza.Pipeline("nl", processors='tokenize,pos', verbose=False), 
       "en": stanza.Pipeline("en", processors='tokenize,pos', verbose=False)}

In [3]:
# read the IATE file
IATE_FILE = os.path.join("P:", "projects", "tbx-data", "iate", "IATE_export_small.tbx")
ref = determinator.TbxDocument().open(IATE_FILE)
# read the lassy file
LASSY_FILE = os.path.join("P:", "projects", "tbx-data", "lassy", "lassy_with_insurance.tbx")
lassy = determinator.TbxDocument().open(LASSY_FILE)

In [4]:
def merge_terms_dict(a, b):
    for key in b.keys():
        if key in a.keys() and a[key]['dc:language']==b[key]['dc:language']:
            for item in b[key]['dc:uri']:
                a[key]['dc:uri'].append(item)
            for item in b[key]['frequency']:
                a[key]['frequency'].append(item)
        else:
            a[key] = {}
            for b_key in b[key].keys():
                a[key][b_key] = b[key][b_key]

In [5]:
# Create an empty TermBase
t = determinator.TbxDocument()
t.generate(params = {"sourceDesc": "TBX file, created via dnb/determinator"})

In [6]:
# termNotes = {"normativeAuthorization": "regulatedTerm", 
#              "authoritative body": "European Union"}
termNotes = {}

names = ['Solvency 2 Directive',
         'Solvency 2 Delegated Acts', 
         'European Climate Law', 
         'Sustainable Finance Disclosure Regulation']

# create terms dictionary of subset of languages
terms = {}

for name in names:
    for language in ['NL', 'EN', 'DE', 'FR']:
        DOC_FILE = os.path.join("P:", "projects", "naf-data", "data", "legislation", name, name+" - "+language+".naf.xml")
        doc = nafigator.NafDocument().open(DOC_FILE)
        if doc.language=="nl":
            nafigator.glue_terms_separated_by_soft_hyphens(doc, "nl", nlp)
            nafigator.glue_terms_separated_by_hard_hyphens(doc, "nl", nlp)
        merge_terms_dict(terms, nafigator.extract_terms(doc, termNotes = termNotes))

In [7]:
name = "legislation"

In [8]:
t.create_tbx_from_terms_dict(terms=terms)
# t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

In [9]:
t.add_termnotes_from_tbx(reference=lassy, params={'number_of_word_components': 5})
# t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

2022-06-15 08:39:42,330 Not completely found: publicatieblad found=[None] pos=['noun']
2022-06-15 08:39:42,330 Not completely found: (besluiten found=[None] pos=['noun']
2022-06-15 08:39:42,330 Not completely found: (herschikking found=[None] pos=['noun']
2022-06-15 08:39:42,330 Not completely found: regio’s found=[None] pos=['noun']
2022-06-15 08:39:42,330 Not completely found: overwegende found=[None] pos=['noun']
2022-06-15 08:39:42,330 Not completely found: co-assurantie found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: rechtsbijstandverze( found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: kering found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: schadeverzeke( found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: euroverzekeringsgroep found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: pees found=[None] pos=['noun']
2022-06-15 08:39:42,345 Not completely found: verzekeringsen fou

2022-06-15 08:39:42,786 Not completely found: overeen5. found=[None] pos=['noun']
2022-06-15 08:39:42,786 Not completely found: forfaitair found=[None] pos=['noun']
2022-06-15 08:39:42,801 Not completely found: semioverheidsinstellingen found=[None] pos=['noun']
2022-06-15 08:39:42,801 Not completely found: onderne3 found=[None] pos=['noun']
2022-06-15 08:39:42,801 Not completely found: ming found=[None] pos=['noun']
2022-06-15 08:39:42,801 Not completely found: b) found=[None] pos=['noun']
2022-06-15 08:39:42,801 Not completely found: lidmaatschapskaart found=[None] pos=['noun']
2022-06-15 08:39:42,832 Not completely found: reciprociteitsovereenkomst found=[None] pos=['noun']
2022-06-15 08:39:42,832 Not completely found: laatstgenoemden found=[None] pos=['noun']
2022-06-15 08:39:42,832 Not completely found: begrafeniskosten found=[None] pos=['noun']
2022-06-15 08:39:42,832 Not completely found: sterfgeval found=[None] pos=['noun']
2022-06-15 08:39:42,848 Not completely found: wirtscha

2022-06-15 08:39:43,115 Not completely found: niet-gestort found=[None] pos=['noun']
2022-06-15 08:39:43,115 Not completely found: gensbestanddeel found=[None] pos=['noun']
2022-06-15 08:39:43,115 Not completely found: toe4. found=[None] pos=['noun']
2022-06-15 08:39:43,115 Not completely found: inge1. found=[None] pos=['noun']
2022-06-15 08:39:43,115 Not completely found: achterstelling found=[None] pos=['noun']
2022-06-15 08:39:43,115 Not completely found: (afwezigheid found=[None] pos=['noun']
2022-06-15 08:39:43,131 Not completely found: herverzekeringsbezwaringen found=[None] pos=['noun']
2022-06-15 08:39:43,131 Not completely found: 3-bestanddelen found=[None] pos=['noun']
2022-06-15 08:39:43,194 Not completely found: ziektekostenverzekeringstechnisch found=[None] pos=['noun']
2022-06-15 08:39:43,272 Not completely found: ziektekostenverzekeringstechnische found=[None] pos=['noun']
2022-06-15 08:39:43,272 Not completely found: gekali4. found=[None] pos=['noun']
2022-06-15 08:39:4

2022-06-15 08:39:43,783 Not completely found: groeps2. found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: herverzekeringsonder-neming found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: reerd found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: voori found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: schuld2 found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: onbeschikbaarheid found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: ringsondernemingen found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: vervat found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: 1resp found=[None] pos=['noun']
2022-06-15 08:39:43,814 Not completely found: autoritei7 found=[None] pos=['noun']
2022-06-15 08:39:43,955 Not completely found: geconsolideerhet found=[None] pos=['noun']
2022-06-15 08:39:43,955 Not completely found: geaggregeerhet found=[None]

2022-06-15 08:39:44,284 Not completely found: дружество found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: selskaber found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: aktiengesellschaft found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: azioni found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: μετοχικ found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: sabiedrība found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: biedrība found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: groothertogdom found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: részvénytársaság found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: szövetkezet found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: székhelyű found=[None] pos=['noun']
2022-06-15 08:39:44,300 Not completely found: limitata found=[None] pos=['noun']

...

2022-06-15 08:41:37,123 Not completely found: dienstverlener eventueel enigerlei found=[('dienst_verlener', '(soort,ev,basis,zijd,stan)', ['dienstverlener']), ('eventueel', '(vrij,basis,zonder)', ['eventueel']), None] pos=['noun', 'adj', 'noun']
2022-06-15 08:41:37,123 Not completely found: enigerlei uitbestede functies found=[None, ('uitbesteed', '(vrij,basis,zonder)', ['uitbestede']), ('functie', '(soort,mv,basis)', ['functies'])] pos=['noun', 'adj', 'noun']
2022-06-15 08:41:37,123 Not completely found: tegenpartij waarschijnlijk een rating found=[('tegen_partij', '(soort,ev,basis,zijd,stan)', ['tegenpartij']), ('waarschijnlijk', '(vrij,basis,zonder)', ['waarschijnlijk']), None, ('rating', '(soort,ev,basis)', ['rating'])] pos=['noun', 'adj', 'det', 'noun']
2022-06-15 08:41:37,123 Not completely found: bestanddeel overeenkomstig dit artikel found=[('bestanddeel', '(soort,ev,basis,onz,stan)', ['bestanddeel']), ('overeenkomstig', '(vrij,basis,zonder)', ['overeenkomstig']), None, ('artik

2022-06-15 08:41:37,374 Not completely found: ondermodule aandelenrisico artikel found=[None, ('aandelenrisico', '(soort,ev,basis,onz,stan)', ['aandelen-', 'risico']), ('artikel', '(soort,ev,basis,onz,stan)', ['artikel'])] pos=['adj', 'noun', 'noun']
2022-06-15 08:41:37,374 Not completely found: ondermodule vastgoedrisico artikel found=[None, ('vastgoedrisico', '(soort,ev,basis,onz,stan)', ['vastgoed-', 'risico']), ('artikel', '(soort,ev,basis,onz,stan)', ['artikel'])] pos=['adj', 'noun', 'noun']
2022-06-15 08:41:37,374 Not completely found: ondermodule spreadrisico artikel found=[None, ('spreadrisico', '(soort,ev,basis,onz,stan)', ['spread-', 'risico']), ('artikel', '(soort,ev,basis,onz,stan)', ['artikel'])] pos=['adj', 'noun', 'noun']
2022-06-15 08:41:37,374 Not completely found: ondermodule marktrisicoconcentraties artikel found=[None, ('markt_risicoconcentratie', '(soort,mv,basis)', ['markt-', 'risico-', 'concentraties']), ('artikel', '(soort,ev,basis,onz,stan)', ['artikel'])] pos=

2022-06-15 08:41:37,578 Not completely found: intergouvernementeel platform found=[None, ('platform', '(soort,ev,basis,onz,stan)', ['platform'])] pos=['noun', 'noun']
2022-06-15 08:41:37,579 Not completely found: punt b) found=[('punt', '(soort,ev,basis,onz,stan)', ['punt']), None] pos=['noun', 'noun']
2022-06-15 08:41:37,579 Not completely found: netto broeikasgasemissies found=[None, ('broeikasgasemissie', '(soort,mv,basis)', ['broeikasgas-', 'emissies'])] pos=['noun', 'noun']
2022-06-15 08:41:37,581 Not completely found: unie overeenkomstig het beginsel found=[('unie', '(soort,ev,basis,zijd,stan)', ['unie']), ('overeenkomstig', '(vrij,basis,zonder)', ['overeenkomstig']), None, ('beginsel', '(soort,ev,basis,onz,stan)', ['beginsel'])] pos=['noun', 'adj', 'det', 'noun']
2022-06-15 08:41:37,581 Not completely found: commissie regelmatig de vooruitgang found=[('commissie', '(soort,ev,basis,zijd,stan)', ['commissie']), ('regelmatig', '(vrij,basis,zonder)', ['regelmatig']), None, ('vooruit

2022-06-15 08:41:38,253 Not completely found: retailbeleggingsproducten found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: priip's found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: "pensioenproduct found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: "pan-europees found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: ibip’s found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: abi found=[None] pos=['noun']
2022-06-15 08:41:38,253 Not completely found: abi’s found=[None] pos=['noun']
2022-06-15 08:41:38,331 Not completely found: hulpbronnenefficiëntie-indicatoren found=[None] pos=['noun']
2022-06-15 08:41:38,331 Not completely found: retailbelegger found=[None] pos=['noun']
2022-06-15 08:41:38,363 Not completely found: beleggingsbeslissingsprocedure found=[None] pos=['noun']
2022-06-15 08:41:38,379 Not completely found: entiteitsniveau found=[None] pos=['noun']
2022-06-15 08:41:38,379 Not c

2022-06-15 08:41:39,030 Not completely found: financiëlemarktdeelnemer artikel found=[None, ('artikel', '(soort,ev,basis,onz,stan)', ['artikel'])] pos=['noun', 'noun']
2022-06-15 08:41:39,053 Not completely found: websites financiëlemarktdeelnemers found=[('web_site', '(soort,mv,basis)', ['websites']), None] pos=['noun', 'noun']
2022-06-15 08:41:39,053 Not completely found: verzekeringstussenpersonen ibpv's found=[('verzekeringstussenpersoon', '(soort,mv,basis)', ['verzekerings-', 'tussenpersonen']), None] pos=['noun', 'noun']
2022-06-15 08:41:39,053 Not completely found: pan-europees persoonlijk pensioenproduct found=[None, ('persoonlijk', '(vrij,basis,zonder)', ['persoonlijk']), ('pensioenproduct', '(soort,ev,basis,onz,stan)', ['pensioen-', 'product'])] pos=['noun', 'adj', 'noun']
2022-06-15 08:41:39,053 Not completely found: "pan-europees persoonlijk pensioenproduct found=[None, ('persoonlijk', '(vrij,basis,zonder)', ['persoonlijk']), ('pensioenproduct', '(soort,ev,basis,onz,stan)',

In [10]:
t.copy_from_tbx(reference=ref)
t.write(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))

In [11]:
# t = determinator.TbxDocument().open(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))
# t.validate(validation_file=os.path.join("..", "data", "dialects", "TBX-DNB.rng")

In [12]:
t = determinator.TbxDocument().open(os.path.join("P:", "projects", "tbx-data", "termbases", name+".tbx"))
t.to_excel(os.path.join("P:", "projects", "tbx-data", "termbases", name+".xlsx"), languages=["en", "nl"])